In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [6]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    y0 = 1.0
    x = [0.9791789663709559,0.8936323663149114]
    frac = 0.8
    Shell_T.Tumor(x[0], frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    Shell_H.Healthy(x[0],frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    print('done')

T 53.392371004926865
H 75.60295376823717
done
T 41.94746013722439
H 62.40345581382408
done
T 31.099191650434076
H 44.96771135546004
done
T 20.869497872618325
H 28.209560514716458
done
T 12.508050858428177
H 16.00039385689085
done
T 6.91774723086882
H 8.551013356434856
done


# Tumor Cell
### (i) CAR_killing
    1/78.94391239956171 == 0.012667221190389747
### (ii) WT killing
    1/40.27411983691734 ==0.02482984119949279

# Healthy Cell
### (i) CAR_killing
    33.209108322965996
### (ii) WT killing
    7.810341768315665

In [8]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(1, 2) # #variables, #objectives)
        self.types[:] = [Real(0.0, 1.0)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        f1 = self.Shell_H.Healthy(x[0], 0.8, 8.93632366e-01, NK_No=NK_No, Tc_No = Tumor_No) # Objective 1
        f2 = self.Shell_T.Tumor(x[0], 0.8, 8.93632366e-01, NK_No=NK_No, Tc_No = Tumor_No) # Objective 2
        solution.objectives[:] = [f1, f2]

In [10]:
for ET_ratio in [1,5,10]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        alpha1 =  [s.variables[0] for s in algorithm.result]
        filename = f'a1_Pfront_Kasumi_Healthy_CAR_NK_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['alpha1','Tumor_lysis', 'Healthy_Lysis'])
        df['alpha1'] = alpha1
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)

x0 [0.22444681]
H 5.66804496176746
T 11.713595311271197
x0 [0.03709775]
H 3.9097162166574884
T 10.550435888691112
x0 [0.77933804]
H 17.678814800064547
T 16.34907879919276
x0 [0.38686998]
H 7.9963913205473025
T 13.053394033474152
x0 [0.36771509]
H 7.665299482885191
T 12.887318772922175
x0 [0.02852964]
H 3.8506527788941924
T 10.505910292395237
x0 [0.58651955]
H 12.397056775671867
T 14.776347791361555
x0 [0.52245275]
H 10.802733957925037
T 14.23143490093407
x0 [0.31055643]
H 6.77403350857404
T 12.400014585726959
x0 [0.44043528]
H 9.006180168265788
T 13.520358011160694
x0 [0.00782791]
H 3.7137901656312478
T 10.399912250632038
x0 [0.22054122]
H 5.622912673962954
T 11.684487525501453
x0 [0.38725626]
H 8.003231748105899
T 13.056751459452421
x0 [0.51754255]
H 10.686938827790925
T 14.189224765151165
x0 [0.61679963]
H 13.198507475894372
T 15.029840389613724
x0 [0.11099876]
H 4.495436611581638
T 10.957891128722618
x0 [0.02682684]
H 3.8390938844599765
T 10.49711065807194
x0 [0.74192274]
H 16.62176